<a href="https://colab.research.google.com/github/sneha4948/Federated-Learning-for-Supply-Chain-Sensor-Data-Processing/blob/main/MLP_Classification_with_Simulated_data_flower_local.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MLP Classification

### Installing FLOWER, other requirements

In [1]:
!pip install -q "flwr[simulation]" flwr-datasets

In [2]:
# you might see a warning after running the command below, this can be ignored
# if you are running this outside Colab, you probably need to adjust the command below
# !pip install torch==1.13.1+cpu torchvision==0.14.1+cpu --extra-index-url https://download.pytorch.org/whl/cpu

In [3]:
!pip install matplotlib

In [51]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.impute import KNNImputer
from sklearn.metrics import mean_squared_error, mean_absolute_error
from scipy.stats import pearsonr
import matplotlib.pyplot as plt
from statsmodels.tsa.api import VAR
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
# Flower Simulation code
import flwr as fl
from flwr.common import Context

In [5]:
# Random seed for reproducibility
np.random.seed(42)

# Parameters: Temperature, Humidity, CO2
n_samples = 1000  # Number of samples

### Dataset Simulation

In [4]:


# Generate random data within given ranges
temperature = np.random.uniform(8, 14, n_samples)
humidity = np.random.uniform(80, 95, n_samples)
co2 = np.random.uniform(600, 2000, n_samples)

# Function to label condition
def label_condition(temp, hum, co2):
    if temp > 12 and hum > 90 and co2 > 1500:
        return "Poor Condition"
    elif (temp > 10 and hum > 85) or (co2 > 1200):
        return "Average Condition"
    else:
        return "Good Condition"

# Create labels
labels = [label_condition(temp, hum, co2) for temp, hum, co2 in zip(temperature, humidity, co2)]

# Create a DataFrame
df = pd.DataFrame({
    'Temperature': temperature,
    'Humidity': humidity,
    'CO2': co2,
    'Condition': labels
})

# Save dataset to CSV
df.to_csv('container_conditions.csv', index=False)
print("Dataset created and saved as container_conditions.csv")


Dataset created and saved as container_conditions.csv


### Data Preprocessing for Classification

In [62]:
# Load dataset
data = pd.read_csv('container_conditions.csv')

# Encode the labels to integers
le = LabelEncoder()
data['Condition'] = le.fit_transform(data['Condition'])

# Split data into features and labels
X = data[['Temperature', 'Humidity', 'CO2']]
y = data['Condition']

# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


## MLP Model

In [94]:
# Creating model function
def create_model():
    model = tf.keras.Sequential([
        tf.keras.layers.InputLayer(shape=(3,)),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(3, activation='softmax')  # 3 output classes
    ])
    model.compile(
        optimizer='adam',
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## Client

In [95]:
# Define Flower client
class ContainerClient(fl.client.NumPyClient):
    def __init__(self, model, X_train, y_train, X_test, y_test):
        self.model = model
        self.X_train = X_train
        self.y_train = y_train
        self.X_test = X_test
        self.y_test = y_test

    def get_parameters(self, config=None):  # Add config argument here
        return self.model.get_weights()

    def fit(self, parameters, config):
        self.model.set_weights(parameters)
        self.model.fit(self.X_train, self.y_train, epochs=1, batch_size=32, verbose=0)
        return self.model.get_weights(), len(self.X_train), {}

    def evaluate(self, parameters, config):
        self.model.set_weights(parameters)
        loss, accuracy = self.model.evaluate(self.X_test, self.y_test)
        return loss, len(self.X_test), {"accuracy": accuracy}


# Correct client_fn signature
def client_fn(context: Context):
    model = create_model()
    return NumPyClient.to_client(ContainerClient(model, X_train, y_train, X_test, y_test))

### FedAvg, FedProx, FedBuff, FedAsync Strategy

In [63]:
from flwr.server.strategy import FedAvg, FedProx
from flwr.server.server import ServerConfig
from flwr.client import NumPyClient


In [64]:
# Define FedBuff strategy
class FedBuffStrategy(FedAvg):
    def __init__(self, buffer_size=2, **kwargs):
        super().__init__(**kwargs)
        self.buffer_size = buffer_size
        self.buffer = []

    def aggregate_fit(self, rnd, results, failures):
        # Buffer the results
        self.buffer.extend(results)
        if len(self.buffer) >= self.buffer_size:
            # Aggregate when buffer is full
            aggregated_parameters, aggregated_metrics = super().aggregate_fit(rnd, self.buffer, failures)
            self.buffer = []  # Reset buffer
            return aggregated_parameters, aggregated_metrics
        else:
            # Return None to indicate that aggregation is not yet performed
            return None, {}


In [65]:
from flwr.server.strategy import Strategy
from flwr.common import Parameters, FitRes, EvaluateRes, Scalar
from typing import Optional, Tuple, List, Dict
import numpy as np

class FedAsyncStrategy(Strategy):
    def __init__(self, eta: float = 0.1):
        self.current_weights: Optional[List[np.ndarray]] = None
        self.eta = eta  # Learning rate for asynchronous updates

    def initialize_parameters(self, client_manager):
        # Initialize global model parameters
        if self.current_weights is None:
            # Create a new model to get initial weights
            model = create_model()
            self.current_weights = model.get_weights()
        parameters = fl.common.weights_to_parameters(self.current_weights)
        return parameters

    def configure_fit(
        self, rnd: int, parameters: Parameters, client_manager
    ) -> List[Tuple[fl.server.client_proxy.ClientProxy, fl.common.FitIns]]:
        # Configure clients to train asynchronously
        config = {}
        # Sample all available clients
        clients = client_manager.all()
        fit_ins = fl.common.FitIns(parameters, config)
        instructions = [(client, fit_ins) for client in clients]
        return instructions

    def configure_evaluate(
        self, rnd: int, parameters: Parameters, client_manager
    ) -> List[Tuple[fl.server.client_proxy.ClientProxy, fl.common.EvaluateIns]]:
        # Optionally, configure clients for evaluation
        # For this example, we will not perform evaluation
        return []

    def aggregate_fit(
        self, rnd: int, results: List[Tuple[fl.server.client_proxy.ClientProxy, FitRes]], failures
    ) -> Tuple[Optional[Parameters], Dict[str, Scalar]]:
        # Update global weights asynchronously
        if not results:
            return None, {}

        for _, fit_res in results:
            client_weights = fl.common.parameters_to_weights(fit_res.parameters)
            if self.current_weights is None:
                self.current_weights = client_weights
            else:
                # Asynchronous update: weighted average
                self.current_weights = [
                    (1 - self.eta) * w_global + self.eta * w_client
                    for w_global, w_client in zip(self.current_weights, client_weights)
                ]

        parameters = fl.common.weights_to_parameters(self.current_weights)
        return parameters, {}

    def aggregate_evaluate(
        self, rnd: int, results: List[Tuple[fl.server.client_proxy.ClientProxy, EvaluateRes]], failures
    ) -> Tuple[Optional[float], Dict[str, Scalar]]:
        # Optionally, aggregate evaluation results
        # Since we're not performing evaluation, return None
        return None, {}

    def evaluate(self, parameters: Parameters):
        # Optional: Implement server-side evaluation if needed
        return None


In [66]:
def start_federated_learning(strategy_name: str, num_rounds: int):
    """Start federated learning with the given number of rounds."""

    # Strategy for the server, using the FedAvg algorithm
    if strategy_name == 'fedavg':
        strategy = FedAvg(
            fraction_fit=1.0,  # Train on all clients each round
            fraction_evaluate=1.0,  # Evaluate on all clients each round
            min_fit_clients=5,  # Minimum number of clients to participate in training
            min_evaluate_clients=5,  # Minimum number of clients to participate in evaluation
            min_available_clients=5,  # Minimum number of available clients needed
        )

    elif strategy_name == 'fedprox':
        strategy = FedProx(
            fraction_fit=1.0,  # Train on all clients each round
            fraction_evaluate=1.0,  # Evaluate on all clients each round
            min_fit_clients=5,  # Minimum number of clients to participate in training
            min_evaluate_clients=5,  # Minimum number of clients to participate in evaluation
            min_available_clients=5,  # Minimum number of available clients needed
            proximal_mu=0.1,  # FedProx proximal term parameter (mu)
    )
    elif strategy_name == 'fedbuff':
        strategy = FedBuffStrategy(
                buffer_size=2,
                fraction_fit=1.0,
                fraction_evaluate=1.0,
                min_fit_clients=2,  # Adjust based on buffer_size
                min_evaluate_clients=2,
                min_available_clients=5,
            )
    elif strategy_name == 'fedasync':
        strategy = FedAsyncStrategy(eta=0.1)


    # Set the number of rounds in ServerConfig
    server_config = ServerConfig(num_rounds=num_rounds)

    # Run the federated learning simulation for `num_rounds`
    fl.simulation.start_simulation(
        client_fn=client_fn,
        num_clients=5,  # Number of clients
        config=server_config,  # Pass the server configuration for the number of rounds
        client_resources={"num_cpus": 1}  # Adjust based on your system resources
    )


## FedAvg

In [101]:
# Run the federated learning simulation for 5 rounds (or any desired number of rounds)
start_federated_learning(strategy_name='fedavg', num_rounds=5)

	Instead, use the `flwr run` CLI command to start a local simulation in your Flower app, as shown for example below:

		$ flwr new  # Create a new Flower app from a template

		$ flwr run  # Run the Flower app in Simulation Mode

	Using `start_simulation()` is deprecated.

            This is a deprecated feature. It will be removed
            entirely in future versions of Flower.
        
INFO :      Starting Flower simulation, config: num_rounds=5, no round_timeout
2024-11-27 16:20:54,787	INFO worker.py:1752 -- Started a local Ray instance.
INFO :      Flower VCE: Ray initialized with resources: {'object_store_memory': 4007909376.0, 'memory': 8015818752.0, 'CPU': 2.0, 'node:__internal_head__': 1.0, 'node:172.28.0.12': 1.0}
INFO :      Optimize your simulation with Flower VCE: https://flower.ai/docs/framework/how-to-run-simulations.html
INFO :      Flower VCE: Resources for each Virtual Client: {'num_cpus': 1}
INFO :      Flower VCE: Creating VirtualClientEngineActorPool with 2 acto

7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7254 - loss: 1.2356  
1/7 ━━━━━━━━━━━━━━━━━━━━ 2s 396ms/step - accuracy: 0.7500 - loss: 0.5977
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7254 - loss: 1.2356  


INFO :      aggregate_evaluate: received 5 results and 0 failures
INFO :      
INFO :      [ROUND 2]
INFO :      configure_fit: strategy sampled 5 clients (out of 5)
INFO :      aggregate_fit: received 5 results and 0 failures
INFO :      configure_evaluate: strategy sampled 5 clients (out of 5)


7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7148 - loss: 2.8046   [repeated 3x across cluster]
1/7 ━━━━━━━━━━━━━━━━━━━━ 2s 387ms/step - accuracy: 0.7500 - loss: 0.5977
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7254 - loss: 1.2356  


INFO :      aggregate_evaluate: received 5 results and 0 failures
INFO :      
INFO :      [ROUND 3]
INFO :      configure_fit: strategy sampled 5 clients (out of 5)
INFO :      aggregate_fit: received 5 results and 0 failures
INFO :      configure_evaluate: strategy sampled 5 clients (out of 5)


...

INFO :      aggregate_evaluate: received 5 results and 0 failures
INFO :      
INFO :      [ROUND 4]
INFO :      configure_fit: strategy sampled 5 clients (out of 5)
INFO :      aggregate_fit: received 5 results and 0 failures
INFO :      configure_evaluate: strategy sampled 5 clients (out of 5)


7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6959 - loss: 2.7839   [repeated 2x across cluster]
1/7 ━━━━━━━━━━━━━━━━━━━━ 2s 359ms/step - accuracy: 0.7188 - loss: 1.4560 [repeated 2x across cluster]
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6719 - loss: 2.7705   [repeated 2x across cluster]


INFO :      aggregate_evaluate: received 5 results and 0 failures
INFO :      
INFO :      [ROUND 5]
INFO :      configure_fit: strategy sampled 5 clients (out of 5)
INFO :      aggregate_fit: received 5 results and 0 failures
INFO :      configure_evaluate: strategy sampled 5 clients (out of 5)


7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6959 - loss: 2.7839   [repeated 3x across cluster]
1/7 ━━━━━━━━━━━━━━━━━━━━ 3s 524ms/step - accuracy: 0.8125 - loss: 0.9616 [repeated 2x across cluster]
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6959 - loss: 2.7839  
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.7305 - loss: 1.5205  
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.7305 - loss: 1.5205  


INFO :      aggregate_evaluate: received 5 results and 0 failures
INFO :      
INFO :      [SUMMARY]
INFO :      Run finished 5 round(s) in 38.90s
INFO :      	History (loss, distributed):
INFO :      		round 1: 1.544061303138733
INFO :      		round 2: 3.847609043121338
INFO :      		round 3: 3.0900609493255615
INFO :      		round 4: 3.233726739883423
INFO :      		round 5: 1.7363842725753784
INFO :      


## FedProx

In [102]:
# Run the federated learning simulation for 5 rounds (or any desired number of rounds)
start_federated_learning(strategy_name='fedprox', num_rounds=5)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
	Instead, use the `flwr run` CLI command to start a local simulation in your Flower app, as shown for example below:

		$ flwr new  # Create a new Flower app from a template

		$ flwr run  # Run the Flower app in Simulation Mode

	Using `start_simulation()` is deprecated.

            This is a deprecated feature. It will be removed
            entirely in future versions of Flower.
        
INFO :      Starting Flower simulation, config: num_rounds=5, no round_timeout
2024-11-27 16:21:47,243	INFO worker.py:1752 -- Started a local Ray instance.
INFO :      Flower VCE: Ray initialized with resources: {'memory': 8016322560.0, 'object_store

7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6357 - loss: 41.6947  


INFO :      aggregate_evaluate: received 5 results and 0 failures
INFO :      
INFO :      [ROUND 2]
INFO :      configure_fit: strategy sampled 5 clients (out of 5)


1/7 ━━━━━━━━━━━━━━━━━━━━ 1s 230ms/step - accuracy: 0.6562 - loss: 33.9789
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6357 - loss: 41.6947  


INFO :      aggregate_fit: received 5 results and 0 failures
INFO :      configure_evaluate: strategy sampled 5 clients (out of 5)


...

INFO :      aggregate_evaluate: received 5 results and 0 failures
INFO :      
INFO :      [ROUND 4]
INFO :      configure_fit: strategy sampled 5 clients (out of 5)
INFO :      aggregate_fit: received 5 results and 0 failures
INFO :      configure_evaluate: strategy sampled 5 clients (out of 5)


1/7 ━━━━━━━━━━━━━━━━━━━━ 3s 587ms/step - accuracy: 0.7188 - loss: 1.2758
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6357 - loss: 2.5089   [repeated 4x across cluster]
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.6768 - loss: 3.1277  


INFO :      aggregate_evaluate: received 5 results and 0 failures
INFO :      
INFO :      [ROUND 5]
INFO :      configure_fit: strategy sampled 5 clients (out of 5)
INFO :      aggregate_fit: received 5 results and 0 failures
INFO :      configure_evaluate: strategy sampled 5 clients (out of 5)


1/7 ━━━━━━━━━━━━━━━━━━━━ 3s 554ms/step - accuracy: 0.7500 - loss: 0.7987 [repeated 2x across cluster]
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6768 - loss: 3.1277   [repeated 3x across cluster]
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6768 - loss: 3.1277  
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6950 - loss: 1.9489  


INFO :      aggregate_evaluate: received 5 results and 0 failures
INFO :      
INFO :      [SUMMARY]
INFO :      Run finished 5 round(s) in 36.70s
INFO :      	History (loss, distributed):
INFO :      		round 1: 41.771297454833984
INFO :      		round 2: 2.361807346343994
INFO :      		round 3: 2.737766981124878
INFO :      		round 4: 3.7461206912994385
INFO :      		round 5: 2.2970986366271973
INFO :      


## FedBuff

In [103]:
# Run the federated learning simulation for 5 rounds (or any desired number of rounds)
start_federated_learning(strategy_name='fedbuff', num_rounds=5)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
	Instead, use the `flwr run` CLI command to start a local simulation in your Flower app, as shown for example below:

		$ flwr new  # Create a new Flower app from a template

		$ flwr run  # Run the Flower app in Simulation Mode

	Using `start_simulation()` is deprecated.

            This is a deprecated feature. It will be removed
            entirely in future versions of Flower.
        
INFO :      Starting Flower simulation, config: num_rounds=5, no round_timeout
2024-11-27 16:22:40,472	INFO worker.py:1752 -- Started a local Ray instance.
INFO :      Flower VCE: Ray initialized with resources: {'CPU': 2.0, 'object_store_memory': 40

7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6768 - loss: 26.7821  
1/7 ━━━━━━━━━━━━━━━━━━━━ 4s 688ms/step - accuracy: 0.7188 - loss: 21.3504
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.6768 - loss: 26.7821  


INFO :      aggregate_evaluate: received 5 results and 0 failures
INFO :      
INFO :      [ROUND 2]
INFO :      configure_fit: strategy sampled 5 clients (out of 5)
INFO :      aggregate_fit: received 5 results and 0 failures
INFO :      configure_evaluate: strategy sampled 5 clients (out of 5)


7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6768 - loss: 5.5773   [repeated 4x across cluster]


INFO :      aggregate_evaluate: received 5 results and 0 failures
INFO :      
INFO :      [ROUND 3]
INFO :      configure_fit: strategy sampled 5 clients (out of 5)
INFO :      aggregate_fit: received 5 results and 0 failures
INFO :      configure_evaluate: strategy sampled 5 clients (out of 5)


...

INFO :      aggregate_fit: received 5 results and 0 failures
INFO :      configure_evaluate: strategy sampled 5 clients (out of 5)


7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.6768 - loss: 3.0812   [repeated 3x across cluster]
1/7 ━━━━━━━━━━━━━━━━━━━━ 2s 371ms/step - accuracy: 0.7188 - loss: 0.9956
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6768 - loss: 2.3193  


INFO :      aggregate_evaluate: received 5 results and 0 failures
INFO :      
INFO :      [ROUND 5]
INFO :      configure_fit: strategy sampled 5 clients (out of 5)
INFO :      aggregate_fit: received 5 results and 0 failures
INFO :      configure_evaluate: strategy sampled 5 clients (out of 5)


7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6719 - loss: 2.0005   [repeated 5x across cluster]
1/7 ━━━━━━━━━━━━━━━━━━━━ 2s 364ms/step - accuracy: 0.7188 - loss: 0.8199
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6719 - loss: 2.0005  


INFO :      aggregate_evaluate: received 5 results and 0 failures
INFO :      
INFO :      [SUMMARY]
INFO :      Run finished 5 round(s) in 32.13s
INFO :      	History (loss, distributed):
INFO :      		round 1: 27.12293243408203
INFO :      		round 2: 5.932343006134033
INFO :      		round 3: 3.4270694255828857
INFO :      		round 4: 2.7804033756256104
INFO :      		round 5: 2.3941431045532227
INFO :      


## FedAsync

In [104]:
# Run the federated learning simulation for 5 rounds (or any desired number of rounds)
start_federated_learning(strategy_name='fedasync', num_rounds=5)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
	Instead, use the `flwr run` CLI command to start a local simulation in your Flower app, as shown for example below:

		$ flwr new  # Create a new Flower app from a template

		$ flwr run  # Run the Flower app in Simulation Mode

	Using `start_simulation()` is deprecated.

            This is a deprecated feature. It will be removed
            entirely in future versions of Flower.
        
INFO :      Starting Flower simulation, config: num_rounds=5, no round_timeout
2024-11-27 16:23:28,698	INFO worker.py:1752 -- Started a local Ray instance.
INFO :      Flower VCE: Ray initialized with resources: {'object_store_memory': 4007421542.0, 

7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6768 - loss: 8.6092  


INFO :      aggregate_evaluate: received 5 results and 0 failures
INFO :      
INFO :      [ROUND 2]
INFO :      configure_fit: strategy sampled 5 clients (out of 5)
INFO :      aggregate_fit: received 5 results and 0 failures
INFO :      configure_evaluate: strategy sampled 5 clients (out of 5)


7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6725 - loss: 1.1726   [repeated 5x across cluster]


INFO :      aggregate_evaluate: received 5 results and 0 failures
INFO :      
INFO :      [ROUND 3]
INFO :      configure_fit: strategy sampled 5 clients (out of 5)
INFO :      aggregate_fit: received 5 results and 0 failures
INFO :      configure_evaluate: strategy sampled 5 clients (out of 5)


...

INFO :      aggregate_fit: received 5 results and 0 failures
INFO :      configure_evaluate: strategy sampled 5 clients (out of 5)


7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6768 - loss: 4.3514   [repeated 4x across cluster]
1/7 ━━━━━━━━━━━━━━━━━━━━ 2s 380ms/step - accuracy: 0.7188 - loss: 2.7052
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6768 - loss: 4.3514  


INFO :      aggregate_evaluate: received 5 results and 0 failures
INFO :      
INFO :      [ROUND 5]
INFO :      configure_fit: strategy sampled 5 clients (out of 5)
INFO :      aggregate_fit: received 5 results and 0 failures
INFO :      configure_evaluate: strategy sampled 5 clients (out of 5)


7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5977 - loss: 1.8573   [repeated 4x across cluster]


INFO :      aggregate_evaluate: received 5 results and 0 failures
INFO :      
INFO :      [SUMMARY]
INFO :      Run finished 5 round(s) in 32.69s
INFO :      	History (loss, distributed):
INFO :      		round 1: 10.643089294433594
INFO :      		round 2: 1.3442435264587402
INFO :      		round 3: 2.8703043460845947
INFO :      		round 4: 4.744094371795654
INFO :      		round 5: 2.031358480453491
INFO :      
